# Load books and descriptions into python

In [1]:
import os
import json

In [7]:
directory="goodreads-data/books"
books = {}

In [8]:
for filename in os.listdir(directory):
    if filename.endswith('.json'):  # Check if the file is a JSON file
        filepath = os.path.join(directory, filename)
        
        # Open and read the JSON file
        with open(filepath, 'r') as file:
            try:
                data = json.load(file)  # Load the JSON data
                print(f"Contents of {filename}:")
                print(data)
                books[data['book_title']] = data['book_description']

            except json.JSONDecodeError as e:
                print(f"Error reading {filename}: {e}")

Contents of 61439040-1984.json:
{'book_id_title': '61439040-1984', 'book_id': '61439040', 'book_title': '1984', 'book_description': "A masterpiece of rebellion and imprisonment where war is peace freedom is slavery and Big Brother is watching. Thought Police, Big Brother, Orwellian - these words have entered our vocabulary because of George Orwell's classic dystopian novel 1984. The story of one man's Nightmare Odyssey as he pursues a forbidden love affair through a world ruled by warring states and a power structure that controls not only information but also individual thought and memory 1984 is a prophetic haunting tale More relevant than ever before 1984 exposes the worst crimes imaginable the destruction of truth freedom and individuality. With a foreword by Thomas Pynchon. This beautiful paperback edition features deckled edges and french flaps a perfect gift for any occasionAlternate cover edition can be found here.", 'book_url': 'https://www.goodreads.com/book/show/61439040-198

## Sucessfully loaded book titles and descriptions

In [10]:
len(books)

105

# Tokenize descriptions

In [13]:
import spacy

# Load the SpaCy English model
nlp = spacy.load('en_core_web_sm')


In [14]:
# Function to process descriptions
def process_description(description):
    
    # Process the text
    doc = nlp(description)
        
    # Extract lemmas of non-stop words
    lemmas = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    
    return lemmas

In [15]:
tokenized_descriptions = {}
for i in books:
    tokenized_descriptions[i] = process_description(books[i])
    

71